In [12]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')
import os

import xml.etree.ElementTree as ET
import csv

DATA_DIR = "/content/drive/MyDrive/Colab_Notebooks/Mahrez_Enterance_Data/Data/"
kml_file = "Dmgisnet_Enterances.kml"

# Define the KML namespace mapping
ns = {'kml': 'http://www.opengis.net/kml/2.2'}

# Parse the KML file
tree = ET.parse(os.path.join(DATA_DIR, kml_file))
root = tree.getroot()

# Prepare a list to hold the extracted data
extracted_data = []

# Iterate over all Placemark elements in the file
for placemark in root.findall(".//kml:Placemark", ns):
    # Find the SchemaData element that holds our target SimpleData elements
    schema_data = placemark.find("kml:ExtendedData/kml:SchemaData", ns)
    if schema_data is None:
        continue  # Skip if no schema data is found

    # Extract each required field using the 'name' attribute
    community = schema_data.find("kml:SimpleData[@name='COMMUNITY_']", ns)
    name_eng = schema_data.find("kml:SimpleData[@name='NAME_ENG']", ns)
    makani_num = schema_data.find("kml:SimpleData[@name='MAKANI_NUM']", ns)
    point_x = schema_data.find("kml:SimpleData[@name='POINT_X']", ns)
    point_y = schema_data.find("kml:SimpleData[@name='POINT_Y']", ns)

    # Get the coordinates from the Point element
    coordinates = placemark.find("kml:Point/kml:coordinates", ns)

    # Retrieve text for each element (default to empty string if missing)
    community_text = community.text if community is not None else ""
    name_eng_text = name_eng.text if name_eng is not None else ""
    makani_num_text = makani_num.text if makani_num is not None else ""
    point_x_text = point_x.text if point_x is not None else ""
    point_y_text = point_y.text if point_y is not None else ""
    coordinates_text = coordinates.text.strip() if (coordinates is not None and coordinates.text) else ""

    # Append the extracted values as a dictionary to our list
    extracted_data.append({
        "NAME_ENG": name_eng_text,
        "MAKANI_NUM": makani_num_text,
        "POINT_X": point_x_text,
        "POINT_Y": point_y_text,
        "COMMUNITY_": community_text,
        "coordinates": coordinates_text
    })

# Define the output CSV file path in your Google Drive.
csv_file = "Dmgisnet_Enterances_extracted_data.csv"
csv_file_path = os.path.join(DATA_DIR, csv_file)

# Write the extracted data to the CSV file.
with open(csv_file_path, mode="w", newline="", encoding="utf-8") as csvfile:
    fieldnames = ["NAME_ENG", "MAKANI_NUM", "POINT_X", "POINT_Y", "COMMUNITY_", "coordinates"]
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

    writer.writeheader()  # Write column headers
    for row in extracted_data:
        writer.writerow(row)

print("CSV file has been written to:", csv_file_path)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
CSV file has been written to: /content/drive/MyDrive/Colab_Notebooks/Mahrez_Enterance_Data/Data/Dmgisnet_Enterances_extracted_data.csv


In [14]:
import os
import pandas as pd

# Read the CSV file into a DataFrame
df = pd.read_csv(csv_file_path)

len_df = len(df)
print("Number of rows in the original DataFrame:", len_df)

# 1. Split the 'coordinates' column into three new columns: coordinate_x, coordinate_y, coordinate_z.
#    We assume each value in 'coordinates' is like "x,y,z".
coords = df['coordinates'].str.split(',', expand=True)
df['coordinate_x'] = coords[0].str.strip()
df['coordinate_y'] = coords[1].str.strip()
df['coordinate_z'] = coords[2].str.strip()

# 2. Process the 'MAKANI_NUM' column:
#    - Check if the value is a string before splitting. If not, assign an empty list.
#    - Strip any extra whitespace from each value.
#    - Use explode() to create a separate row for each MAKANI_NUM instance.
df['MAKANI_NUM'] = df['MAKANI_NUM'].apply(
    lambda x: [num.strip() for num in x.split(',')] if isinstance(x, str) else []
)
df = df.explode('MAKANI_NUM')

# Optionally, you can drop the original 'coordinates' column if it's no longer needed:
# df.drop(columns=['coordinates'], inplace=True)

modified_csv_file = 'Dmgisnet_Enterances_extracted_data_modified.csv'
modified_csv_file_path = os.path.join(DATA_DIR, modified_csv_file)
df.to_csv(modified_csv_file_path, index=False)

len_df = len(df)
print("Number of rows in the new DataFrame:", len_df)
print("CSV file has been written to:", modified_csv_file_path)

Number of rows in the original DataFrame: 70142
Number of rows in the new DataFrame: 108289
CSV file has been written to: /content/drive/MyDrive/Colab_Notebooks/Mahrez_Enterance_Data/Data/Dmgisnet_Enterances_extracted_data_modified.csv
